In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
import gc

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.models import Sequential

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(42)

In [5]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')
sample_submission = pd.read_csv('./dts/sample_submission.csv')

In [6]:
#레이블(Delay)을 제외한 결측값이 존재하는 변수들을 학습 데이터의 최빈값으로 대체합니다
NaN_col = ['Origin_State','Destination_State','Airline','Estimated_Departure_Time', 'Estimated_Arrival_Time','Carrier_Code(IATA)','Carrier_ID(DOT)']

for col in NaN_col:
    mode = train[col].mode()[0]
    train[col] = train[col].fillna(mode)
    
    if col in test.columns:
        test[col] = test[col].fillna(mode)
print('Done.')

Done.


In [7]:
#질적 변수들을 수치화합니다
qual_col = ['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Airline', 'Carrier_Code(IATA)', 'Tail_Number']

for i in qual_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])
print('Done.')

Done.


In [8]:
#레이블이 없는 데이터들을 제거합니다
train = train.dropna()
print('Done.')

Done.


In [9]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train.loc[:, 'Delay_num'] = train['Delay'].apply(lambda x: to_number(x, column_number))
print('Done.')

Done.


In [10]:
train_x = train.drop(columns=['ID', 'Delay', 'Delay_num'])
train_y = train['Delay_num']
test_x = test.drop(columns=['ID'])

In [12]:
train_x.shape,test_x.shape

((255001, 17), (1000000, 17))

In [33]:
train_y = train_y.apply(lambda x: x-1)

In [34]:
x_train, x_val, y_train, y_val = train_test_split(train_x, train_y, test_size=.25, random_state=42, stratify=train_y)

In [55]:
model = Sequential()

model.add(Dense(1000, activation='relu', input_shape=(None, 17)))
model.add(Dense(500, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(125, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))
model.add(Activation("sigmoid"))

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, None, 1000)        18000     
                                                                 
 dense_18 (Dense)            (None, None, 500)         500500    
                                                                 
 dense_19 (Dense)            (None, None, 250)         125250    
                                                                 
 dense_20 (Dense)            (None, None, 125)         31375     
                                                                 
 dense_21 (Dense)            (None, None, 32)          4032      
                                                                 
 dense_22 (Dense)            (None, None, 4)           132       
                                                                 
 dense_23 (Dense)            (None, None, 1)          

In [56]:
model.compile(optimizer='adam', loss='binary_crossentropy')

# hist = model.fit(x_train, y_train, epochs=200, batch_size=32000, validation_data=(x_val, y_val), class_weight={1:.18, 0:.82})
hist = model.fit(x_train, y_train, epochs=200, batch_size=32000, validation_data=(x_val, y_val))

Epoch 1/200
6/6 [==============================] - 2s 263ms/step - loss: 121.0946 - val_loss: 20.3469
Epoch 2/200
6/6 [==============================] - 1s 228ms/step - loss: 64.6287 - val_loss: 2.4889
Epoch 3/200
6/6 [==============================] - 1s 238ms/step - loss: 37.9370 - val_loss: 11.4979
Epoch 4/200
6/6 [==============================] - 1s 230ms/step - loss: 23.2575 - val_loss: 2.8937
Epoch 5/200
6/6 [==============================] - 1s 229ms/step - loss: 17.6237 - val_loss: 35.4242
Epoch 6/200
6/6 [==============================] - 1s 237ms/step - loss: 21.6170 - val_loss: 7.8691
Epoch 7/200
6/6 [==============================] - 1s 225ms/step - loss: 15.5542 - val_loss: 15.8395
Epoch 8/200
6/6 [==============================] - 1s 234ms/step - loss: 13.3117 - val_loss: 11.7541
Epoch 9/200
6/6 [==============================] - 1s 234ms/step - loss: 12.9213 - val_loss: 3.9904
Epoch 10/200
6/6 [==============================] - 1s 229ms/step - loss: 12.8147 - val_loss: 

KeyboardInterrupt: 

In [37]:
result = model.predict(test_x)

31250/31250 [==============================] - 11s 363us/step


In [40]:
pd.DataFrame(result)

,0
0,4.961738e-17
1,1.891652e-13
2,3.555940e-15
3,1.600588e-15
4,6.429205e-12
...,...
999995,1.236099e-14
999996,4.848558e-16
999997,1.216191e-13
999998,5.977423e-17
